## Section 3: Impacts

In [1]:
#Necessary packages
from ecmwf.opendata import Client
from ipyleaflet import Choropleth, Map
import ipywidgets as widgets
from IPython.display import display
from Magics import macro as magics
from Magics.macro import * 
import numpy as np
import pandas as pd
import requests
import rioxarray as rxr
import xarray as xr

In [ ]:
#Parameters to be imported from the main widget
initial_lat_lon = (38, -90)
final_lat_lon = (60, 90)

In [ ]:
#Variable initialization
min_lat = initial_lat_lon[0]
min_lon = initial_lat_lon[1]

max_lat = final_lat_lon[0]
max_lon = final_lat_lon[1]

## Population density layer

## Hazard layers

### Coastal hazard

The return period `rp` could be provided by the user through a widget or we could just select a specific return period and use just that one

In [ ]:
def dwnl_coastalhaz(rp):
    """
    Downloads coastal hazard raster maps from The World Bank Data Catalog service
    by providing the hazard return period
    
    rp: str
        Return period.
        Can be one of the following: 5yr, 10yr, 50yr, 100yr, 250yr, 500yr, 1000yr.

    Data source: https://datacatalog.worldbank.org/search/dataset/0038579/Global-coastal-flood-hazard
    """
    links = {
            "5yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0005m.tif",
            "10yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0010m.tif",
            "50yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0050m.tif",
            "100yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0100m.tif",
            "250yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0250m.tif",
            "500yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0500m.tif",
            "1000yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp1000m.tif",
            }
    tif = requests.get(links[rp])
    with open(f"data/impacts/coastalhaz_{rp}.tif", "wb") as tiffile:
            tiffile.write(tif.content)
            #need to compress the tif file when downloading it
            print("Download complete")

def load_coastalhaz(rp):
    """
    Loads coastal hazard raster maps downloaded through dwnl_coastalhaz
    by providing the return period. Returns an xarray
    
    rp: str
        Return period.
        Can be one of the following: 5yr, 10yr, 50yr, 100yr, 250yr, 500yr, 1000yr.
    """
    path = f"data/impacts/coastalhaz_{rp}.tif"
    coh = rxr.open_rasterio(path)
    return(coh)

def plot_coastalhaz(coh, resample = True, cut = False):
    # resample the raster to be lighter (less accuracy)
    # make the user able to decide if resampling or not
    if resample:
        pass
    
    # cut the raster in an area near the tropical cyclone position
    if cut:
        pass

    # plot the raster in an interactive map with ipyleaflet
    pass

In [ ]:
#Download the 50yr coastal hazard raster map
dwnl_coastalhaz("50yr")
#Load the 50yr coastal hazard raster map as an xarray
coh = load_coastalhaz("50yr")
print(coh)

### Cyclone hazard

The return period `rp` could be provided by the user through a widget or we could just select a specific return period and use just that one

In [ ]:
def dwnl_cyclonehaz(rp):
    """
    Downloads cyclone hazard raster maps from The World Bank Data Catalog service
    by providing the hazard return period

    rp: str
        Return period.
        Can be one of the following: 50yr, 100yr, 250yr, 500yr, 1000yr.
    
    Data source: https://datacatalog.worldbank.org/search/dataset/0038577/Global-cyclone-hazard
    """
    links = {
            "50yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr50_int1_Ai5zJup.tif",
            "100yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr100_int1_afg5XsL.tif",
            "250yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr250_int1.tif",
            "500yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr500_int1.tif",
            "1000yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr1000_int1.tif",
            }
    tif = requests.get(links[rp])
    with open(f"data/impacts/cyclonehaz_{rp}.tif", "wb") as tiffile:
            tiffile.write(tif.content)
            print("Download complete")

def load_cyclonehaz(rp):
    """
    Loads coastal hazard raster maps downloaded through dwnl_coastalhaz
    by providing the return period. Returns an xarray

    rp: str
        Return period.
        Can be one of the following: 50yr, 100yr, 250yr, 500yr, 1000yr.
    """
    path = f"data/impacts/cyclonehaz_{rp}.tif"
    cyh = rxr.open_rasterio(path)
    return(cyh)

def plot_cyclonehaz(cyh):
    # these rasters are lighter, shouldn't be a problem to plot not resampled

    pass

In [ ]:
#Download the 50yr cyclone hazard raster map
dwnl_cyclonehaz("50yr")
#Load the 50yr cyclone hazard raster map as an xarray
cyh = load_cyclonehaz("50yr")

### World Risk Index
From the data of the World Risk Index, retrieve the expositions to coastal floodings, tsunamis and sea level rise of each country.

**Issue to solve**: check values higher than 100 in the columns and if so remove it or find a way to obtain the percentage. China does not have percentage but number of people. Check the world risk index 2021 and Trend, maybe there there's no error

In [7]:
def dwnl_riskidx():
    """
    Downloads the World Risk Index for 2022

    Data source: https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73
    """
    links = {
     #     "meta": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/9ab4cdc6-9682-45a6-8314-db5e0d49e7a7/download/worldriskindex-meta.xlsx",
         "data": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/1b47b40c-f746-427c-bbf8-8caa157e03da/download/worldriskindex-2022.csv",
     #     "trend": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/9ae68502-6011-4276-a99d-118bb2826323/download/worldriskindex-trend.csv"
    }
    for id in links:
        f = requests.get(links[id])
        if id == "meta":
             fformat = "xlsx"
        else:
             fformat = "csv"
        with open(f"data/impacts/worldriskindex22_{id}.{fformat}", "wb") as file:
                file.write(f.content)
        print("World Risk Index ", id, " downloaded")

def load_riskidx():
     """
     Load data in a DataFrame and keep only columns with the annual percentage of people of that
     country exposed to severe tsunamis, severe coastal floods and sea level rise.
     """
     csv = pd.read_csv("data/impacts/worldriskindex22_data.csv")
     cols = ["Country", "ISO3", "EI_02c_Base", "EI_03c_Base", "EI_07a_Base"]
     csv = csv.loc[:, cols]
     cols = ["Country", "ISO3", "Tsunamis", "Coastal_floods", "Sea_level_rise"]
     csv.columns = cols
     return(csv)

def plot_riskidx(var, csv = None):
    # https://ipyleaflet.readthedocs.io/en/latest/layers/choropleth.html
    # https://coderzcolumn.com/tutorials/data-science/choropleth-maps-using-ipyleaflet
    #Load risk data
    if csv is None: csv = load_riskidx()
    #Load world countries boundaries
    #from: https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/
    f = requests.get("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/world-administrative-boundaries/exports/geojson?lang=en&timezone=Europe%2FBerlin")
    geo_json_data = json.loads(f.content)
    for d in geo_json_data["features"]:
          d["iso"] = d["properties"]["iso3"]
    mapping  = dict(zip(csv["ISO3"].str.strip(), csv[var]))
    for d in geo_json_data["features"]:
        if d["iso"] not in mapping:
            mapping[d["iso"]] = 0 #check if nan is possible
    m = Map(zoom=2)
    layer = Choropleth(
                    geo_data = geo_json_data,
                    choro_data = mapping,
                    # colormap="blues",
                    style = {'fillOpacity': 1.0, "color":"black"},
                    key_on = "iso")
    m.add_layer(layer)
    m.layout.height="700px"
    return(m)

In [8]:
m = plot_riskidx("Tsunamis")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [9]:
m = plot_riskidx("Coastal_floods")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [10]:
m = plot_riskidx("Sea_level_rise")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Combined map

Plot all the layers above in a unique ipyleaflet